<a href="https://colab.research.google.com/github/rahafbader/T5/blob/main/YOLO_Practical_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [1]:
!pip install opencv-python


In [2]:
!pip install ultralytics


## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import cv2
import numpy as np
from ultralytics import YOLO
model = YOLO('/content/drive/MyDrive/YOLO_Model.pt')



## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [5]:
video_path = '/content/YOLO_Video.mp4'
video = cv2.VideoCapture(video_path)



## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [6]:
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
fps = video.get(cv2.CAP_PROP_FPS)


## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [7]:
output_path = '/content/drive/My Drive/YOLO_Video_Output.mp4'
VIDEO_CODEC = "mp4v"
output_video = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*VIDEO_CODEC), fps, (width, height))

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

In [8]:
zones = {
    'upper': (1056, 0, 1171, 356),
    'lower': (672, 831, 787, 1080),
    'left_side': (288, 378, 576, 540),
    'right_side': (1190, 572, 1536, 712)
}

object_count = {'upper': 0, 'lower': 0, 'left_side': 0, 'right_side': 0}
tracked_objects = {'upper': set(), 'lower': set(), 'left_side': set(), 'right_side': set()}

while video.isOpened():
    frame_captured, current_image = video.read()
    if not frame_captured:
        break

    tracking_result = model.track(current_image, persist=True, verbose=False, device=0)

    bounding_boxes = tracking_result[0].boxes.xywh.cpu().numpy()
    ids_list = tracking_result[0].boxes.id.int().cpu().tolist()
    class_types = tracking_result[0].boxes.cls.cpu().tolist()
    confidence_levels = tracking_result[0].boxes.conf.cpu().tolist()

    for index, bbox in enumerate(bounding_boxes):
        x_center, y_center, box_width, box_height = map(int, bbox[:4])
        x1, y1 = int(x_center - box_width / 2), int(y_center - box_height / 2)
        x2, y2 = int(x_center + box_width / 2), int(y_center + box_height / 2)
        identifier = ids_list[index]
        confidence_score = confidence_levels[index]
        class_label = class_types[index]

        for region, (x_min, y_min, x_max, y_max) in zones.items():
            if y_min < y_center < y_max and x_min < x_center < x_max:
                if identifier not in tracked_objects[region]:
                    object_count[region] += 1
                    tracked_objects[region].add(identifier)

        cv2.rectangle(current_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(current_image, f'{class_label} ID:{identifier} Conf:{confidence_score:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    for region, (x_min, y_min, x_max, y_max) in zones.items():
        cv2.rectangle(current_image, (x_min, y_min), (x_max, y_max), (255, 0, 0), 2)

    cv2.putText(current_image, f'{object_count["upper"]}', (int(width * 0.55), int(height * 0.32)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
    cv2.putText(current_image, f'{object_count["lower"]}', (int(width * 0.39), int(height * 0.80)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
    cv2.putText(current_image, f'{object_count["left_side"]}', (int(width * 0.28), int(height * 0.38)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
    cv2.putText(current_image, f'{object_count["right_side"]}', (int(width * 0.62), int(height * 0.56)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

    output_video.write(current_image)

video.release()
output_video.release()
cv2.destroyAllWindows()


## Save and Submit Your Work

https://drive.google.com/file/d/1zKIG7gie12U1N53jHjxArPGqpSmoosZE/view?usp=drive_link

Submit both the notebook and the output video